In [1]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
import numpy as np
import constantes as cons

lista=[
'AGLDUSDT',
'PHBUSDT',
'FTMUSDT',
'SRMUSDT',
'KAVAUSDT',
'EDUUSDT',
'CRVUSDT',
'XRPUSDT',
'XTZUSDT',
'PENDLEUSDT',
'BELUSDT',
'ARKMUSDT',
'RDNTUSDT',
'KNCUSDT',
'PERPUSDT',
'RUNEUSDT',
'EOSUSDT'
]
lista_filtrada = []

lista=md.lista_de_monedas ()
#lista=['PHBUSDT']

for symbol in lista:    
    try:
        #data = md.estrategia_santa(symbol,tp_flag = True)
        data = md.estrategia_triangulos(symbol, tp_flag = True, print_lines_flag = False)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = False)
        if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):
            lista_filtrada.append(symbol)
            print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KNCUSDT - Return [%]: 6.04% - # Trades: 6 - Profit Factor: 11.085006629281377 - Win Rate [%]: 83.33333333333334
ARKMUSDT - Return [%]: 1.31% - # Trades: 2 - Profit Factor: 2.428774105661098 - Win Rate [%]: 50.0
BELUSDT - Return [%]: 0.68% - # Trades: 2 - Profit Factor: 3.0170962455050416 - Win Rate [%]: 50.0
AGLDUSDT - Return [%]: 2.15% - # Trades: 2 - Profit Factor: 6.955240737600756 - Win Rate [%]: 50.0
STXUSDT - Return [%]: 2.36% - # Trades: 2 - Profit Factor: nan - Win Rate [%]: 100.0
FTMUSDT - Return [%]: 0.44% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:458: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


XRPUSDT - Return [%]: 1.59% - # Trades: 4 - Profit Factor: 5.109757415146956 - Win Rate [%]: 50.0
SRMUSDT - Return [%]: 3.8% - # Trades: 3 - Profit Factor: nan - Win Rate [%]: 100.0
SUSHIUSDT - Return [%]: 1.83% - # Trades: 5 - Profit Factor: 4.729369832014142 - Win Rate [%]: 60.0
SXPUSDT - Return [%]: 0.4% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
ALICEUSDT - Return [%]: 3.72% - # Trades: 8 - Profit Factor: 4.830722865773053 - Win Rate [%]: 62.5
PENDLEUSDT - Return [%]: 2.06% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
MATICUSDT - Return [%]: 2.51% - # Trades: 4 - Profit Factor: nan - Win Rate [%]: 100.0
EOSUSDT - Return [%]: 1.51% - # Trades: 3 - Profit Factor: nan - Win Rate [%]: 100.0
CRVUSDT - Return [%]: 3.52% - # Trades: 6 - Profit Factor: 3.1661379777481313 - Win Rate [%]: 50.0
OMGUSDT - Return [%]: 1.2% - # Trades: 3 - Profit Factor: 8.711336166832321 - Win Rate [%]: 66.66666666666666


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:458: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


EDUUSDT - Return [%]: 1.35% - # Trades: 2 - Profit Factor: nan - Win Rate [%]: 100.0


In [4]:
md.dibuja_patrones_triangulos (data,933) 

0.9625689680241795 -1.0


linea superior. X: [916. 929. 944.] - y: [0.5798     0.5777     0.57527692]
linea inferior. X: [917. 925. 930. 945.] - y: [0.56746124 0.57200078 0.57483798 0.58334961]
